### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2023
quarter = 2

today = date.today()
today

datetime.date(2023, 8, 19)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2023 AND quarter <= 2) 
OR (year = 2023-1 AND quarter >= 2+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp.shape

(211, 5)

In [4]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(231, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct.sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
140,RS,2023,Q2,253583,1569,252014,"16,062.08"
71,GRAMMY,2023,Q2,306321,-16061,322382,"2,007.23"
171,SUPEREIF,2023,Q2,340669,56986,283683,497.81
115,NOBLE,2023,Q2,660067,125165,534902,427.36
111,MINT,2023,Q2,8797837,-4225107,13022944,308.23


In [6]:
sql = """
DELETE FROM yr_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

181

In [7]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
rcds = df_ins.values.tolist()
len(rcds)

211

In [8]:
for rcd in rcds:
    print(rcd)

['ACE', 2023, 'Q2', 1166351, 1485354, -319003, -21.48, 698]
['ADVANC', 2023, 'Q2', 27332445, 25853429, 1479016, 5.72, 6]
['AH', 2023, 'Q2', 1983193, 1175297, 807896, 68.74, 9]
['AIE', 2023, 'Q2', -186609, 402015, -588624, -146.42, 720]
['AIMIRT', 2023, 'Q2', 693288, 566114, 127174, 22.46, 669]
['AIT', 2023, 'Q2', 473640, 556639, -82999, -14.91, 11]
['AJ', 2023, 'Q2', -536546, 422747, -959293, -226.92, 12]
['AMATA', 2023, 'Q2', 1944361, 2171767, -227406, -10.47, 21]
['ANAN', 2023, 'Q2', -115177, -796635, 681458, 85.54, 23]
['AOT', 2023, 'Q2', -1336443, -16784073, 15447630, 92.04, 24]
['AP', 2023, 'Q2', 5596465, 5329148, 267317, 5.02, 25]
['ASIAN', 2023, 'Q2', 532125, 1048954, -516829, -49.27, 36]
['ASK', 2023, 'Q2', 1541463, 1367575, 173888, 12.72, 38]
['ASP', 2023, 'Q2', 536435, 578771, -42336, -7.31, 40]
['ASW', 2023, 'Q2', 1059831, 871576, 188255, 21.6, 728]
['AWC', 2023, 'Q2', 4976058, 3075541, 1900517, 61.79, 699]
['BA', 2023, 'Q2', 1302119, -8985803, 10287922, 114.49, 45]
['BAM', 

In [9]:
sql = """INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)

INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [10]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [11]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [12]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
115,NOBLE,2023,Q2,660067,125165,534902,427.36
111,MINT,2023,Q2,8797837,-4225107,13022944,308.23
37,CENTEL,2023,Q2,1169423,-672616,1842039,273.86
33,BKI,2023,Q2,4203494,-3805995,8009489,210.44
50,CRC,2023,Q2,8220076,2819760,5400316,191.52


In [13]:
criteria_2 = df_ins.q_amt_p > 300000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
50,CRC,2023,Q2,8220076,2819760,5400316,191.52
154,SIRI,2023,Q2,6262643,2191557,4071086,185.76
127,PRM,2023,Q2,2791103,1067852,1723251,161.38
108,MC,2023,Q2,847819,364688,483131,132.48
142,SABUY,2023,Q2,1365128,596382,768746,128.90


In [14]:
criteria_3 = df_ins.Pct > 5.0
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
140,RS,2023,Q2,253583,1569,252014,"16,062.08"
71,GRAMMY,2023,Q2,306321,-16061,322382,"2,007.23"
171,SUPEREIF,2023,Q2,340669,56986,283683,497.81
115,NOBLE,2023,Q2,660067,125165,534902,427.36
111,MINT,2023,Q2,8797837,-4225107,13022944,308.23


In [15]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
50,CRC,2023,Q2,8220076,2819760,5400316,191.52
154,SIRI,2023,Q2,6262643,2191557,4071086,185.76
127,PRM,2023,Q2,2791103,1067852,1723251,161.38
108,MC,2023,Q2,847819,364688,483131,132.48
142,SABUY,2023,Q2,1365128,596382,768746,128.90
31,BH,2023,Q2,6378422,2799073,3579349,127.88
177,TFG,2023,Q2,3359137,1543647,1815490,117.61
75,HANA,2023,Q2,2393988,1117296,1276692,114.27
28,BEM,2023,Q2,3115812,1474508,1641304,111.31
124,PLANB,2023,Q2,784671,375646,409025,108.89


In [16]:
df_ins.loc[df_ins_criteria,cols].sort_values(['name'],ascending=[True])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
1,ADVANC,2023,Q2,27332445,25853429,1479016,5.72
2,AH,2023,Q2,1983193,1175297,807896,68.74
4,AIMIRT,2023,Q2,693288,566114,127174,22.46
10,AP,2023,Q2,5596465,5329148,267317,5.02
12,ASK,2023,Q2,1541463,1367575,173888,12.72
14,ASW,2023,Q2,1059831,871576,188255,21.60
15,AWC,2023,Q2,4976058,3075541,1900517,61.79
19,BAY,2023,Q2,32562537,27998571,4563966,16.30
20,BBL,2023,Q2,36649261,27306314,9342947,34.22
24,BDMS,2023,Q2,13032398,11252429,1779969,15.82
